## Azure Language Service Pre Built Capabilities and Offerings

In [ ]:
%pip install azure-ai-textanalytics==5.3.0 azure-identity==1.26.0b1 python-dotenv==1.2.1

### Setting up the Environment Variables

In [ ]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv
import os
import json

load_dotenv()
endpoint = os.getenv("AZURE_LANGUAGE_ENDPOINT")
key = os.getenv("AZURE_LANGUAGE_KEY")

### Creating the Azure Language Service Client

In [ ]:
client = TextAnalyticsClient(endpoint=endpoint, credential=AzureKeyCredential(key))

### Preparing the Documents for Analysis

In [ ]:
documents = {}

for review_file in os.listdir("reviews/"):
    with open(os.path.join("reviews", review_file), "r", encoding="utf-8") as f:
        documents[review_file] = f.read()

### Extract Key Phrases

In [ ]:
response = client.extract_key_phrases(list(documents.values()))
for idx, doc in enumerate(response):
    doc_name = list(documents.keys())[idx]
    if not doc.is_error:
        print(f"Key phrases in '{doc_name}':")
        for phrase in doc.key_phrases:
            print(f"  - {phrase}")
    else:
        print(f"Error in '{doc_name}': {doc.error.code} - {doc.error.message}")

### Find Linked Entities

In [ ]:
response = client.recognize_linked_entities(list(documents.values()))
for idx, doc in enumerate(response):
    doc_name = list(documents.keys())[idx]
    if not doc.is_error:
        print(f"Linked entities in '{doc_name}':")
        for entity in doc.entities:
            print(f"  - Name: {entity.name}, ID: {entity.data_source_entity_id}, URL: {entity.url}, Source: {entity.data_source}")
    else:
        print(f"Error in '{doc_name}': {doc.error.code} - {doc.error.message}")

### Named Entity Recognition

In [ ]:
response = client.recognize_entities(list(documents.values()))
for idx, doc in enumerate(response):
    doc_name = list(documents.keys())[idx]
    if not doc.is_error:
        for entity in doc.entities:
            print(f"Entity in '{doc_name}': {entity.text}, Category: {entity.category}, Subcategory: {entity.subcategory}")
    else:
        print(f"Error in '{doc_name}': {doc.error.code} - {doc.error.message}")

### Personally Identifiable Information (PII) Recognition

In [ ]:
response = client.recognize_pii_entities(list(documents.values()))
for idx, doc in enumerate(response):
    doc_name = list(documents.keys())[idx]
    if not doc.is_error:
        for entity in doc.entities:
            print(f"PII Entity in '{doc_name}': {entity.text}, Type: {entity.category}")
    else:
        print(f"Error in '{doc_name}': {doc.error.code} - {doc.error.message}")

### Sentiment Analysis

In [ ]:
response = client.analyze_sentiment(list(documents.values()), show_opinion_mining=True)

for idx, doc in enumerate(response):
    if not doc.is_error:
        doc_name = list(documents.keys())[idx]
        print(f"\nText: {documents[doc_name]}")
        print(f"Overall Sentiment: {doc.sentiment}")
        print(f"Positive Score: {doc.confidence_scores.positive:.2f}")
        print(f"Neutral Score: {doc.confidence_scores.neutral:.2f}")
        print(f"Negative Score: {doc.confidence_scores.negative:.2f}")
        for sentence in doc.sentences:
            print(f"\tSentence: {sentence.text}")
            print(f"\tSentence Sentiment: {sentence.sentiment}")
            print(f"\tScores: Pos={sentence.confidence_scores.positive:.2f}, Neu={sentence.confidence_scores.neutral:.2f}, Neg={sentence.confidence_scores.negative:.2f}")
    else:
        print(f"\nError in document {idx}: {doc.error}")